## Prompt Engineering Playground

#### This notebook contains examples of how to effectively use prompting strategies in code, as well as what to look out for. We will be using Ollama as the LLM. 

#### Important tips
     1) LLM outputs have no built-in mechanism for output verification - be skeptical! 
     2) LLMs are *not* replacements for sound software principles and algorithmic analysis. 
     3) LLMs are great for assisting in code documentation, but use the outputs as templates and rought drafts
     4) The distinction between prompt strategies is not always clear-cut, many overlap and/or build off others.

#### Prompt strategies
     1) I/O prompting
     2) Prompt templating & chaining
     3) Few-shot prompting
     4) Retrieval-Augmented Generation (RAG)
     5) Chain-of-Thought (CoT)
     6) Automated Prompt Engineering (APE) & Meta-prompting



In [1]:
!pip install numpy
!pip install pandas
!pip install ollama
!pip install scikit-learn
!pip install torch
!pip install transformers 
#!pip install huggingface-hub 


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\622267\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\622267\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\622267\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\622267\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\622267\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\622267\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
#Downloading Ollama
import os 
import ollama

ollama.list() #lists available ollama models
ollama.pull("gemma2:2b") #we will use the lightweight gemma2 with 2B parameters for this notebook
response = ollama.generate(model = 'gemma2:2b',
                           prompt = 'explain LLMs')

print(response['response'])

#This cell should take between 4 - 5 min to run on a CPU (GPU)

Let's break down what Large Language Models (LLMs) are. 

**What are LLMs?**

Imagine a computer program that can hold and understand human language in a way similar to how we do.  That's essentially the core idea behind LLMs! These powerful AI systems, built on deep learning, are trained on massive datasets of text (think books, articles, code). 

**How Do LLMs Work?**

1. **Data is King:** LLMs learn by analyzing enormous amounts of text data. This process helps them develop a vast understanding of language structure, grammar, and meaning.
2. **Transformers at the Helm:** A key component in many LLMs is the "Transformer" architecture.  It excels at processing sequences like words or sentences (think sentence-level analysis) and capturing relationships between these elements. 
3. **Predicting the Next Word:**  LLMs are essentially sophisticated predictive models. They analyze a given text sequence and try to predict the most probable next word based on what has come before. By repeati

In [5]:
#I/O prompt

#### This is the simplest case - you simply ask the LLM to complete a task and record the output. 
#### NOT SUITABLE for complex use cases (otherwise, we wouldn't bother with prompt engineering)

prompt = "write me a python function to find the minimum of an array"

llm_output = ollama.generate(model = 'gemma2:2b',
                             prompt = prompt)['response'] #this will output the code snippet

print(llm_output)

```python
def min_array(arr):
  """
  Finds the minimum value in an array.

  Args:
    arr: The array to find the minimum of.

  Returns:
    The minimum value in the array. 
  """

  if not arr:
    return None  # Return None if the array is empty

  min_value = arr[0]  # Initialize min_value with the first element
  for value in arr:
    if value < min_value:
      min_value = value 
  return min_value


```

**Explanation:**

1. **Function Definition (`def min_array(arr):`)**: This defines a function named `min_array` that takes one argument, `arr`, which represents the array we want to examine.

2. **Empty Array Check (`if not arr: return None`)**:  It checks if the array is empty using `not arr`. If it's empty, it returns `None` to avoid errors. This makes the code more robust. 

3. **Initialization (`min_value = arr[0]`)**: The code initializes a variable `min_value` with the value of the first element in the array using `arr[0]`.  

4. **Iterating through the Array (`for value 

##### Check - is the program valid? Have you checked your inputs into the function? Is there a more efficient way to write the piece of code? 

In [ ]:
## Prompt templating & chaining

#### Prompt template - rather than having a fixed string, you format it with some other value(s) before passing to an LLM. This can take many forms but here is one for reference
character = "Spongebob"
template = f"Explain in one sentence who {character} is"
llm_output = ollama.generate(model = 'gemma2:2b',
                             prompt = template)

## Prompt chaining -   input -> LLM -> output -> LLM -> output -+ ... 
### Prompt chaining is where you break down your problem into simpler tasks and chain together the outputs to solve the overall larger goal. 
### Note that we can combine multiple templated prompts into a prompt chain 

project_description = 'custom memory allocator'
prompt_1 = 'explain the tradeoffs when using the GCC compiler for C++ vs. Clang'


compiler_tradeoffs = ollama.generate(model = 'gemma2:2b',
                                     prompt = prompt_1)['response'] #get the compiler tradeoffs description 

prompt_2 = f"""Write me a recommendation for which compiler to use for my C++ project: {project_description},
 considering the differences between Clang and GCC explained here: {compiler_tradeoffs}""" 


recommendation = ollama.generate(model = 'gemma2:2b',
                                 prompt = prompt_2)['response']


print(recommendation)

In [15]:
##Few-shot prompting

#### Few-shot prompting is where we provide several curated examples in addition to a prompt as context.

examples = [("This is great!","positive"),("This is terrible!","negative"),("This is just okay","neutral")]
formatted_examples = '\n'.join(['{} : {}'.format(examples[i][0],examples[i][1]) for i in range(len(examples))])

prompt = f"""I want you to classify text into positive, negative, or netural sentiment. 
Please output only the sentiment. Examples are below:\n{formatted_examples}"""



print(prompt)

ollama.generate(model = 'gemma2:2b',
                prompt = prompt)['response']

I want you to classify text into positive, negative, or netural sentiment. 
Please output only the sentiment. Examples are below:
This is great! : positive
This is terrible! : negative
This is just okay : neutral
